In [7]:
import os

# Mudar o diretório para a raiz do projeto
os.chdir(r"C:\Users\DELL\Documents\bank-analyst-project")
print("Diretório atual:", os.getcwd())




Diretório atual: C:\Users\DELL\Documents\bank-analyst-project


In [8]:
import pandas as pd

customers = pd.read_csv("data/raw/customers.csv")
tx = pd.read_csv("data/raw/transactions.csv", parse_dates=["txn_date"])

display(customers.head())
display(tx.head())


,customer_id,name,age,gender,city
0,1,Customer 1,31,M,São Paulo
1,2,Customer 2,25,F,Rio de Janeiro
2,3,Customer 3,46,M,Rio de Janeiro
3,4,Customer 4,27,F,Belo Horizonte
4,5,Customer 5,35,F,Rio de Janeiro


,txn_id,customer_id,txn_date,txn_amount,txn_type,channel
0,1,722,2025-02-21 21:37:02.494141,3768.69,payment,branch
1,2,773,2025-09-11 21:37:02.494162,2338.04,payment,atm
2,3,998,2025-01-05 21:37:02.494167,4294.48,payment,app
3,4,555,2025-05-24 21:37:02.494169,2643.92,payment,app
4,5,350,2025-05-08 21:37:02.494172,2438.96,payment,branch


In [9]:
customers.info()
tx['txn_type'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  1000 non-null   int64 
 1   name         1000 non-null   object
 2   age          1000 non-null   int64 
 3   gender       1000 non-null   object
 4   city         1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


txn_type
payment       1695
withdrawal    1666
deposit       1639
Name: count, dtype: int64

In [10]:
tx = tx.dropna(subset=['customer_id','txn_date','txn_amount'])
tx['txn_amount'] = pd.to_numeric(tx['txn_amount'], errors='coerce').fillna(0)


In [11]:
tx['month'] = tx['txn_date'].dt.to_period('M').dt.to_timestamp()

monthly = (tx.groupby(['customer_id','month'])
             .agg(txn_amount_sum=('txn_amount','sum'),
                  txn_count=('txn_amount','count'),
                  avg_txn_value=('txn_amount','mean'))
             .reset_index())

monthly.to_csv("data/processed/monthly_agg.csv", index=False)
monthly.head()


,customer_id,month,txn_amount_sum,txn_count,avg_txn_value
0,1,2025-01-01,1338.75,1,1338.75
1,1,2025-03-01,3924.79,1,3924.79
2,1,2025-05-01,491.06,1,491.06
3,2,2024-10-01,4471.74,1,4471.74
4,2,2024-12-01,2073.17,1,2073.17


In [12]:
percentil = monthly['txn_amount_sum'].quantile(0.995)
monthly = monthly[monthly['txn_amount_sum'] <= percentil]


In [13]:
monthly_full = monthly.merge(customers, on='customer_id', how='left')
monthly_full.head()


,customer_id,month,txn_amount_sum,txn_count,avg_txn_value,name,age,gender,city
0,1,2025-01-01,1338.75,1,1338.75,Customer 1,31,M,São Paulo
1,1,2025-03-01,3924.79,1,3924.79,Customer 1,31,M,São Paulo
2,1,2025-05-01,491.06,1,491.06,Customer 1,31,M,São Paulo
3,2,2024-10-01,4471.74,1,4471.74,Customer 2,25,F,Rio de Janeiro
4,2,2024-12-01,2073.17,1,2073.17,Customer 2,25,F,Rio de Janeiro
